In [45]:
import os
from collections import Counter
import numpy as np
import pandas as pd

In [46]:
INP_PATH = "./../inp/"
train_df = pd.read_csv(os.path.join(INP_PATH, "train.csv"))
test_df = pd.read_csv(os.path.join(INP_PATH, "test.csv"))
sub_df = pd.read_csv(os.path.join(INP_PATH, "sample_submission.csv"))

In [47]:
train_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [48]:
train_df.nunique()

id                                       140700
Name                                        422
Gender                                        2
Age                                          43
City                                         98
Working Professional or Student               2
Profession                                   64
Academic Pressure                             5
Work Pressure                                 5
CGPA                                        331
Study Satisfaction                            5
Job Satisfaction                              5
Sleep Duration                               36
Dietary Habits                               23
Degree                                      115
Have you ever had suicidal thoughts ?         2
Work/Study Hours                             13
Financial Stress                              5
Family History of Mental Illness              2
Depression                                    2
dtype: int64

In [49]:
def map_sleep_hours(response):
    label = -1
    if response == "Less Than 5 hours":
        label = 0
    elif response == "5-6 hours":
        label = 1
    elif response == "7-8 hours":
        label = 2
    elif response == "More than 8 hours":
        label = 3

    return label

def map_diet(response):
    label = -1
    if response == "Unhealthy":
        label = 0
    elif response == "Moderate":
        label = 1
    elif response == "Healthy":
        label = 2
    
    return label
    pass

def preprocess(df):
    df = df.copy()
    # drop the id col
    df.drop(columns='id', inplace=True)

    # convert name to counts
    df['Name'] = df["Name"].map(Counter(df["Name"]))
    
    # Female as 0, Male as 1
    df["Gender"] = df["Gender"].map({"Female": 0, "Male": 1})

    # convert City to counts
    df["City"] = df["City"].map(Counter(df["City"]))

    # working professional or student: binary col
    df["Working Professional or Student"] = df["Working Professional or Student"].map({
        "Working Professional": 1,
        'Student': 0
    })

    # profession to counts
    df['Profession'] = df['Profession'].map(Counter(df['Profession']))

    # # fill academic pressure as -1 for all working professionals
    df['Academic Pressure'] = df['Academic Pressure'].fillna(-1)

    # same for Work Pressue: Combine these columns in future
    df['Work Pressure'] = df['Work Pressure'].fillna(-1)

    # same for CGPA
    df['CGPA'] = df['CGPA'].fillna(-1)
    df['Study Satisfaction'] = df['Study Satisfaction'].fillna(-1)
    df['Job Satisfaction'] = df['Job Satisfaction'].fillna(-1)
    
    df['Sleep Duration'] = df["Sleep Duration"].apply(map_sleep_hours)
    df['Dietary Habits'] = df['Dietary Habits'].apply(map_diet)

    # ignore degree col for now, high cardinality
    df = df.drop(columns="Degree")

    df["Have you ever had suicidal thoughts ?"] = df["Have you ever had suicidal thoughts ?"].map({
        "Yes": 1, "No": 0
    })

    df["Family History of Mental Illness"] = df["Family History of Mental Illness"].map({
        "Yes": 1, "No": 0
    })

    # financial stress NaN?? Dear lord.
    df["Financial Stress"] = df["Financial Stress"].fillna(-1)

    assert df.isnull().sum().sum() == 0, "stil some NaN left"
    return df

In [50]:
processed_train_df = preprocess(train_df)
processed_test_df = preprocess(test_df)

In [51]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [64]:
skf = StratifiedKFold(5, shuffle=True, random_state=2024)

oof_preds = np.zeros(len(processed_train_df))
test_preds = np.zeros(len(processed_test_df))

for model, weight in [(XGBClassifier(), 0.05), (LGBMClassifier(), 0.15), (CatBoostClassifier(verbose=False), 0.8)]:
    print(f"Model: {type(model)}")
    model_preds = np.zeros(len(processed_test_df))
    for fold, (train_idx, val_idx) in enumerate(skf.split(processed_train_df, processed_train_df['Depression'])):
        x_train = processed_train_df.iloc[train_idx].drop(columns='Depression')
        y_train = processed_train_df.iloc[train_idx]['Depression']
        x_val = processed_train_df.iloc[val_idx].drop(columns='Depression')
        y_val = processed_train_df.iloc[val_idx]['Depression']

        model.fit(x_train, y_train)

        y_pred = model.predict(x_val)
        print(f"{fold} : {accuracy_score(y_true=y_val, y_pred=y_pred)}")
        oof_preds[val_idx] += y_pred * weight

        y_pred_test = model.predict(processed_test_df)
        model_preds += y_pred_test / 5
    
    test_preds += model_preds * weight

print("Blend: ", accuracy_score(y_true=processed_train_df['Depression'], y_pred=oof_preds.astype('int')))

Model: <class 'xgboost.sklearn.XGBClassifier'>
0 : 0.9386638237384506
1 : 0.937953091684435
2 : 0.9374911158493248
3 : 0.9392679459843639
4 : 0.9367093105899076
Model: <class 'lightgbm.sklearn.LGBMClassifier'>
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
0 : 0.9401208244491827
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
1 : 0.9381663113006397
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
2 : 0.9388415067519545
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
3 : 0.9391968727789624
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
4 : 0.9372423596304194
Model: <class 'catboost.core.CatBoostClassifier'>
0 : 0.9407604832977967
1 : 0.9381307746979389
2 : 0.9386993603411514
3 : 0.9401918976545842
4 : 0.9380597014925374
Blend:  0.9384577114427861


In [65]:
sub_df['Depression'] = test_preds.astype('int')
sub_df['Depression'].value_counts(normalize=True)

Depression
0    0.856951
1    0.143049
Name: proportion, dtype: float64

In [63]:
sub_df.to_csv('./../op/baseline_blend.csv', index=False)